CHANGE LATER:
- dir_output (in first cell under 2) User Input

# Import and Settings

## 1) Import & Initial Set-Ups, definitions, etc.

In [1]:
# RUN THIS IMPORT CELL TWICE!!! in the first go, it will throw an error (because of Solara)
import solara
import solara.lab

import os
import time
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

plt.close('all')
import dfm_tools as dfmt
from xugrid.plot import line
import hydrolib.core.dflowfm as hcdfm 
import xarray as xr
import pandas as pd
import numpy as np

from ipyleaflet import GeoJSON, Map, basemaps, Polyline, Rectangle, CircleMarker, LayerGroup, LegendControl, AntPath, WidgetControl # , ImageOverlay
from shapely.geometry import LineString
from branca.colormap import linear
import ipywidgets as widgets
import datetime
from pathlib import Path

from shapely.geometry import LineString
import geopandas as gpd
import json

import subprocess
import glob
import shutil

m = Map(center=(55.5, 3), zoom=2, scroll_wheel_zoom=True,basemap=basemaps.OpenStreetMap.Mapnik)

tab_layers = {
    'UserInput': LayerGroup(),
    'GridGen': LayerGroup(),
    'Bnd': LayerGroup(),
    'Forcing': LayerGroup(),
    'Obs': LayerGroup(),
    'mdu': LayerGroup(),
    'run': LayerGroup(),
    'visu1': LayerGroup(),
    'visu2': LayerGroup(),
    'visu3': LayerGroup()
}

current_layer_group = solara.reactive(tab_layers['UserInput'])

tab_controls = solara.reactive({})
control_update_signal = solara.reactive(0)
current_control = solara.reactive(None)

userinputdone = solara.reactive(False) 
gridgendone = solara.reactive(False); grid_generated = solara.reactive(False); grid_refined = solara.reactive(False)
bnddone = solara.reactive(False); tidalbnddone = solara.reactive(False); downloadGTSMdone = solara.reactive(False)
forcingdone = solara.reactive(False); genextforcingdone = solara.reactive(False)#; downloadERA5done = solara.reactive(False)
obsdone = solara.reactive(False) 
mdudone = solara.reactive(False) 
rundone = solara.reactive(False) 

# # >>>>>>>>>>>>> when just running visualisation, here some pre-settings to avoid running everything <<<<<<<<<<<<<<<
# model_name = solara.reactive("Humber_delta")
# dir_output = solara.reactive("C:\\Users\\santjer\\OneDrive - Stichting Deltares\\Documents\\IRISCC\\model") 
# file_nc_his = os.path.join(dir_output.value, f"DFM_OUTPUT_{model_name.value}", f"{model_name.value}_his.nc")
# file_nc_map = os.path.join(dir_output.value, f"DFM_OUTPUT_{model_name.value}", f"{model_name.value}_map.nc")
# crs = 'EPSG:4326'; layer = None; raster_res = 0.5; umag_clim = None; scale = 15

# All Tabs

## 2) User Input

In [2]:
####################################### User Input ########################################
model_name = solara.reactive("Model_Name")
continuous_update = solara.reactive(True)
model_resolution_options = ["0.05", "0.5"]
dx = solara.reactive("0.05")
lat_min = solara.reactive(0.0); lat_max = solara.reactive(0.0); lon_min = solara.reactive(0.0); lon_max = solara.reactive(0.0)
rectangle = Rectangle(bounds=((lat_min.value, lon_min.value), (lat_max.value, lon_max.value)), color="black", fill_opacity=0) # , weight=1)
def update_rectangle():
    if rectangle in current_layer_group.value.layers:
        current_layer_group.value.remove_layer(rectangle)
    rectangle.bounds = ((lat_min.value, lon_min.value), (lat_max.value, lon_max.value))
    current_layer_group.value.add_layer(rectangle)

date_min = solara.reactive(datetime.date(2022, 11, 1))
date_max = solara.reactive(datetime.date(2022, 11, 3))
ref_date = solara.reactive(datetime.date(2022, 1, 1))

dir_output = solara.reactive("C:\\") # ("C:\\Users\\santjer\\OneDrive - Stichting Deltares\\Documents\\IRISCC\\test_loschen")
def makedir():
    global dir_output_data
    dir_path = Path(dir_output.value)  
    os.makedirs(dir_path, exist_ok=True)
    dir_output_data = dir_path / 'data'
    os.makedirs(dir_output_data, exist_ok=True)
    userinputdone.set(True)
    
overwrite = False # used for downloading of forcing data. Always set to True when changing the domain
crs = 'EPSG:4326' # coordinate reference system

In [3]:
@solara.component
def Tab_User_Input():
    solara.use_effect(update_rectangle, dependencies=[lat_min.value, lat_max.value, lon_min.value, lon_max.value]) # RESET THE RECTANGLE: Model area
    
    with solara.Card("User Input", style={"width": "100%", "padding": "10px"}):
        solara.Markdown("""**Note to the user**: In this notebook we use publicly available data 
                        from Copernicus Programme of the European Union. To access this data you 
                        need to create accounts at 
                        [Copernicus Marine Service](https://data.marine.copernicus.eu/register)
                        and the [Climate Data Store](https://cds.climate.copernicus.eu/profile).
                        Do not forget to accept the CDS license agreement.""")
        solara.InputText("Model Name (avoid spaces)", value=model_name, continuous_update=continuous_update.value)
        solara.InputText("Output directory", value=dir_output, continuous_update=True)
        solara.Button(label="Select/Create directory",on_click=makedir)

        solara.Markdown("Area of Model & Resolution:")
        solara.InputFloat("Latitude minimum", value=lat_min, continuous_update=True)
        solara.InputFloat("Latitude maximum", value=lat_max, continuous_update=True)
        solara.InputFloat("Longitude minimum", value=lon_min, continuous_update=True)
        solara.InputFloat("Longitude maximum", value=lon_max, continuous_update=True)     
        solara.Select(label="Model Resolution", value=dx, values=model_resolution_options) # Model Resolution in x-direction

        solara.Markdown("Date selection:")
        solara.Text("Select min date:"); solara.lab.InputDate(date_min)
        solara.Text("Select max date:"); solara.lab.InputDate(date_max)
        if date_max.value < date_min.value:
            solara.Markdown("**Warning**: The end date cannot be earlier than the start date.", 
                            style={"color": "red"})
        ref_date = date_min

        with solara.Row(justify="end"):     
            solara.Button(label="Go to Step 2.", on_click=lambda: selected_tab.set('GridGen'),disabled=not userinputdone.value)
    
# Tab_User_Input()

## 3) Grid Generation, Refinement and Bathymetry

In [4]:
# if pli and net.nc files already exist:
# dir_grid = solara.reactive("C:\\") # ("C:\\Users\\santjer\\OneDrive - Stichting Deltares\\Documents\\IRISCC\\model")
dir_grid_pli = solara.reactive("C:\\")
dir_grid_netnc = solara.reactive("C:\\")

import_grid_status = solara.reactive("Idle")
def import_grid():
    global xu_grid_uds, poly_file, netfile
    import_grid_status.set("Importing")

    poly_file = os.path.join(dir_grid_pli.value)
    with open(poly_file, 'r') as file:
        lines = file.readlines()
    pli_content = [[float(value) for value in line.split()] for line in lines[2:]]
    line_geom = LineString([(lon, lat) for lon, lat in pli_content])
    gdf = gpd.GeoDataFrame(geometry=[line_geom], crs="EPSG:4326")
    gdf_json = gdf.to_json()
    imported_bnd = GeoJSON(data=json.loads(gdf_json),style={"color": "red", "weight": 2},name="Boundaries")

    netfile = os.path.join(dir_grid_netnc.value)
    xu_grid_uds = dfmt.open_partitioned_dataset(netfile)
    plt.ioff(); mpl_linecollection = line(xu_grid_uds.grid); plt.ion()
    segments = mpl_linecollection.get_segments()
    line_geometries = [LineString(segment) for segment in segments]
    gdf = gpd.GeoDataFrame(geometry=line_geometries, crs="EPSG:4326")
    gdf_json = gdf.to_json()
    import_grid_layer = GeoJSON(data=json.loads(gdf_json),style={"color": "blue", "weight": 1},name="Grid Lines")

    for layer in list(current_layer_group.value.layers):
        if isinstance(layer, GeoJSON) and layer.name == "Grid Lines":
            current_layer_group.value.remove_layer(layer)
        if isinstance(layer, GeoJSON) and layer.name == "Boundaries":
            current_layer_group.value.remove_layer(layer)
    current_layer_group.value.add_layer(import_grid_layer) 
    current_layer_group.value.add_layer(imported_bnd) 

    import_grid_status.set("Done")
    gridgendone.set(True) 



plot_bathy_files_status = solara.reactive("Idle")
def plot_bathymetry_files():
    plot_bathy_files_status.set("Plotting")

    bathy_data = xu_grid_uds.mesh2d_node_z.values
    colormap = linear.viridis.scale(round(np.nanmin(bathy_data),1), round(np.nanmax(bathy_data),1))
    scatter_layer = LayerGroup()
    for la, lo, ba in zip(xu_grid_uds.mesh2d_node_z.lat.values, xu_grid_uds.mesh2d_node_z.lon.values, bathy_data):
        if np.isnan(ba):
            continue
        color = colormap(ba)
        marker = CircleMarker(
            location=(la, lo),
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        )
        scatter_layer.add_layer(marker)   
    plot_bathy_files_status.set("Done")
    current_layer_group.value.add_layer(scatter_layer)
    
    legend_html = widgets.HTML(value=colormap._repr_html_())
    legend_control = WidgetControl(widget=legend_html, position='bottomright')
    tab_controls.value["GridGen"] = legend_control
    control_update_signal.set(control_update_signal.value + 1)



In [5]:
# if starting from fresh:
# Grid Generation -------------------------------------------------------------------------------------------------------------------------------------------
grid_status = solara.reactive("Idle")
grid_visible = solara.reactive(False) 
def generate_grid():
    global mk_object, grid_layer, poly_file, gdf
    grid_status.set("Generating")
    # generate spherical regular grid
    mk_object = dfmt.make_basegrid(lon_min.value, lon_max.value, lat_min.value, lat_max.value, dx=float(dx.value), dy=float(dx.value)*1/np.cos(np.radians(lat_min.value)), crs=crs) # IF dy IS INPUT: dx=float(dx.value), dy=float(dy.value), crs=crs) # PREVIOUSLY (dxy): dx=float(dxy.value), dy=float(dxy.value), crs=crs)
    # transform grid data
    mesh = mk_object.mesh2d_get(); node_x = mesh.node_x; node_y = mesh.node_y; edge_nodes = mesh.edge_nodes 
    line_geometries = [LineString([(node_x[start], node_y[start]), (node_x[end], node_y[end])]) for start, end in zip(edge_nodes[::2], edge_nodes[1::2])]
    gdf = gpd.GeoDataFrame(geometry=line_geometries, crs="EPSG:4326")
    # add grid to map
    gdf_json = gdf.to_json()
    grid_layer = GeoJSON(data=json.loads(gdf_json),style={"color": "blue", "weight": 2},name="Grid Lines")
    # generate boundaries
    bnd_gdf = dfmt.generate_bndpli_cutland(mk=mk_object, res='h', buffer=0.01)
    bnd_gdf_interp = dfmt.interpolate_bndpli(bnd_gdf, res=0.03)
    # add red boundaries to map
    bnd_gdf_json = bnd_gdf_interp.to_json()
    bnd_layer = GeoJSON(data=json.loads(bnd_gdf_json),style={"color": "red", "weight": 2},name="Boundaries")
    grid_status.set("Completed")
    # return grid_layer, bnd_layer
    for layer in list(current_layer_group.value.layers):
        if isinstance(layer, GeoJSON) and layer.name == "Grid Lines":
            current_layer_group.value.remove_layer(layer)
        if isinstance(layer, GeoJSON) and layer.name == "Boundaries":
            current_layer_group.value.remove_layer(layer)
    current_layer_group.value.add_layer(grid_layer)
    current_layer_group.value.add_layer(bnd_layer)
    grid_visible.set(True)
    # generate plifile
    pli_polyfile = dfmt.geodataframe_to_PolyFile(bnd_gdf_interp, name=f'{model_name.value}_bnd')
    poly_file = os.path.join(dir_output.value, f'{model_name.value}.pli')
    pli_polyfile.save(poly_file)


# Grid refinement -------------------------------------------------------------------------------------------------------------------------------------------
grid_refinement_options = ["300", "3000"]
min_edge_size = solara.reactive("3000")

grid_refine_export_status = solara.reactive("Idle")
def refine_export_grid():
    global xu_grid_uds, netfile, illegalcells_gdf
    grid_refine_export_status.set("InProcess")
    
    # Refine Grid --------------
    if bathy_choice.value == "Deltares":
        file_nc_bathy = "https://opendap.deltares.nl/thredds/dodsC/opendap/deltares/Delft3D/netcdf_example_files/GEBCO_2022/GEBCO_2022_coarsefac08.nc"
        data_bathy = xr.open_dataset(file_nc_bathy).elevation
    elif bathy_choice.value == "NOAA":
        file_nc_bathy = "https://www.ngdc.noaa.gov/thredds/dodsC/global/ETOPO2022/30s/30s_surface_elev_netcdf/ETOPO_2022_v1_30s_N90W180_surface.nc"
        data_bathy = xr.open_dataset(file_nc_bathy).z
    # else: # NOT IMPLEMENTED
    # subset to area of interest
    data_bathy_sel = data_bathy.sel(lon=slice(lon_min.value-1, lon_max.value+1), lat=slice(lat_min.value-1, lat_max.value+1))
    dfmt.refine_basegrid(mk=mk_object, data_bathy_sel=data_bathy_sel, min_edge_size=min_edge_size.value)
    # PLOT
    # transform grid data
    mesh = mk_object.mesh2d_get(); node_x = mesh.node_x; node_y = mesh.node_y; edge_nodes = mesh.edge_nodes 
    line_geometries = [LineString([(node_x[start], node_y[start]), (node_x[end], node_y[end])]) for start, end in zip(edge_nodes[::2], edge_nodes[1::2])]
    gdf = gpd.GeoDataFrame(geometry=line_geometries, crs="EPSG:4326")
    # add grid to map
    gdf_json = gdf.to_json()
    grid_ref_layer = GeoJSON(data=json.loads(gdf_json),style={"color": "blue", "weight": 2},name="Grid Lines Refined")

    # remove all layers
    for layer in list(current_layer_group.value.layers):
        current_layer_group.value.remove_layer(layer)

    # Remove Land --------------
    # remove land with GSHHS coastlines
    dfmt.meshkernel_delete_withcoastlines(mk=mk_object, res='h')
    # derive illegalcells geodataframe
    illegalcells_gdf = dfmt.meshkernel_get_illegalcells(mk=mk_object)
    # PLOT
    # transform grid data
    mesh = mk_object.mesh2d_get(); node_x = mesh.node_x; node_y = mesh.node_y; edge_nodes = mesh.edge_nodes 
    line_geometries = [LineString([(node_x[start], node_y[start]), (node_x[end], node_y[end])]) for start, end in zip(edge_nodes[::2], edge_nodes[1::2])]
    gdf = gpd.GeoDataFrame(geometry=line_geometries, crs="EPSG:4326")
    # add grid to map
    gdf_json = gdf.to_json()
    grid_wolandcells_layer = GeoJSON(data=json.loads(gdf_json),style={"color": "blue", "weight": 2},name="Grid Lines woLand")    
    current_layer_group.value.add_layer(grid_wolandcells_layer) # m.value.add_layer(grid_wolandcells_layer)
    grid_visible.set(True)

    # Convert to xugrid --------
    xu_grid_uds = dfmt.meshkernel_to_UgridDataset(mk=mk_object, crs=crs)
    # interpolate bathymetry onto the grid
    data_bathy_interp = data_bathy_sel.interp(lon=xu_grid_uds.obj.mesh2d_node_x, lat=xu_grid_uds.obj.mesh2d_node_y)
    xu_grid_uds['mesh2d_node_z'] = data_bathy_interp.clip(max=10)
    xu_grid_uds.obj.mesh2d_node_z.encoding["_FillValue"] = 1e20
    # write xugrid grid to netcdf
    netfile = os.path.join(dir_output.value, f'{model_name.value}_net.nc')
    xu_grid_uds.ugrid.to_netcdf(netfile)

    grid_refine_export_status.set("Done")
    

# Plot Bathymetry ---------------------------------------------------------------------------------------------------------------------------------------
bathymetry_options = ["Deltares", "NOAA", "Own data (not implemented)"]
bathy_choice = solara.reactive("Deltares")

plot_bathy_status = solara.reactive("Idle")
legend_bathy = []
def plot_bathymetry():
    global legend_bathy
    gdf_plot = xu_grid_uds.mesh2d_node_z.ugrid.to_geodataframe(name="bathy")
    plot_bathy_status.set("Plotting")
    min_bathy, max_bathy = gdf_plot["bathy"].min(), gdf_plot["bathy"].max()
    colormap = linear.viridis.scale(round(min_bathy,2), round(max_bathy,2))

    scatter_layer = LayerGroup()  # Layer for all markers
    for _, row in gdf_plot.iterrows():
        color = colormap(row["bathy"])  # Map value to color
        marker = CircleMarker(
            location=(row["mesh2d_node_y"], row["mesh2d_node_x"]),
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        )
        scatter_layer.add_layer(marker)
    
    plot_bathy_status.set("Done")
    current_layer_group.value.add_layer(scatter_layer)

    legend_html = widgets.HTML(value=f"<b>Elevation</b><br>{colormap._repr_html_()}") # widgets.HTML(value=colormap._repr_html_())
    legend_control = WidgetControl(widget=legend_html, position='bottomright')
    tab_controls.value["GridGen"] = legend_control
    control_update_signal.set(control_update_signal.value + 1)
    

# Check Orthogonality ---------------------------------------------------------------------------------------------------------------------------------------
ortho_status = solara.reactive("Idle")
def orthogonality_check():
    ortho_status.set("Checking")
    ortho = mk_object.mesh2d_get_orthogonality()
    ortho_vals = ortho.values
    ortho_vals[ortho_vals==ortho.geometry_separator] = 0
    print('mk.mesh2d_get_orthogonality()')
    print(mk_object.mesh2d_get_orthogonality().values.max())
    xu_grid_uds['ortho'] = xr.DataArray(ortho_vals, dims=xu_grid_uds.grid.edge_dimension)
    ortho_status.set("Done")
    gridgendone.set(True) 


In [6]:
@solara.component
def Tab_Grid():    

    with solara.lab.Tabs(): 
            with solara.lab.Tab("Import Existing"):
                with solara.Card("Grid Generation", style={"width": "100%", "padding": "10px"}):
                    solara.Markdown("Follow this in case grid data already exists (i.e. '.pli' and '_net.nc') for the model case.")
                    solara.InputText("Grid file (.pli)", value=dir_grid_pli, continuous_update=True)
                    path_pli = Path(dir_grid_pli.value); pli_valid = False
                    if dir_grid_pli.value:
                        if not dir_grid_pli.value.endswith(".pli"):
                            solara.Error(label='Error: File must end with ".pli"', text=False, dense=True, outlined=True, icon=False)
                        elif not path_pli.exists():
                            solara.Error("File does not exist.")
                        else:
                            pli_valid = True
                    
                    solara.InputText("Grid file (_net.nc)", value=dir_grid_netnc, continuous_update=True)
                    path_netnc = Path(dir_grid_netnc.value); netnc_valid = False
                    if dir_grid_netnc.value:
                        if not dir_grid_netnc.value.endswith("_net.nc"):
                            solara.Error(label='Error: File must end with "_net.nc"', text=False, dense=True, outlined=True, icon=False)
                        elif not path_netnc.exists():
                            solara.Error("File does not exist.")
                        else:
                            netnc_valid = True
                    
                    solara.Markdown("**Importing:**")
                    solara.Button(label="Import Grid",on_click=import_grid, continuous_update=True, disabled=not (pli_valid and netnc_valid))
                    if import_grid_status.value == "Importing":
                        solara.Markdown("Importing... Please wait.")
                    elif import_grid_status.value == "Done":
                        solara.Markdown("Done!")
                    
                    solara.Markdown("**Bathymetry:**")
                    solara.Button(label="Plot Bathymetry",on_click=plot_bathymetry_files, continuous_update=True, disabled=not (pli_valid and netnc_valid))
                    if plot_bathy_files_status.value == "Plotting":
                        solara.Markdown("Preparing plot... Please wait. This might take a couple of minutes.")
                    elif plot_bathy_files_status.value == "Done":
                        solara.Markdown("Done!")
                        
                with solara.Row(justify="end"):     
                    solara.Button(label="Go to Step 3.", on_click=lambda: selected_tab.set('Bnd'),disabled=not gridgendone.value)


                
            with solara.lab.Tab("Create New"):
                with solara.Card("Grid Generation", style={"width": "100%", "padding": "10px"}):
                    solara.Button(label="Generate Grid",on_click=lambda: (generate_grid(), grid_generated.set(True)),continuous_update=True) # solara.Button(label="Generate Grid",on_click=generate_grid, continuous_update=True)
                    if grid_status.value == "Generating":
                        solara.Markdown("Generating grid... Please wait.")
                    elif grid_status.value == "Completed":
                        solara.Markdown("Grid generation completed successfully!")
                    
                    solara.Markdown("**Grid Refinement:**")
                    solara.Select(label="Grid Refinement", value=min_edge_size, values=grid_refinement_options)
            
                    solara.Button(label="Refine and Export Grid",on_click=lambda: (refine_export_grid(), grid_refined.set(True)),continuous_update=True,disabled=not grid_generated.value) # solara.Button(label="Refine and Export Grid",on_click=refine_export_grid,continuous_update=True,disabled=not grid_generated.value)
                    if not grid_generated.value:
                        solara.Markdown("Grid needs to be generated first.")
                    if grid_refine_export_status.value == "InProcess":
                        solara.Markdown("Refining grid... Please wait.")
                    elif grid_refine_export_status.value == "Done":
                        solara.Markdown("Grid refinement and exportation completed successfully!") 
                    
                    solara.Markdown("**Bathymetry:**")
                    solara.Markdown("""Note, if 'Own data' is chosen: You can download your own full 
                                    resolution cutout from [gebco data]( https://download.gebco.net) 
                                    (use a buffer of e.g. 1 degree).""")
                    solara.Select(label="Bathymetry", value=bathy_choice, values=bathymetry_options)
                    
                    solara.Button(label="Plot Bathymetry",on_click=plot_bathymetry, continuous_update=True,disabled=not grid_refined.value) # solara.Button(label="Plot Bathymetry",on_click=lambda: (plot_bathymetry(), grid_refined.set(True)), continuous_update=True,disabled=not grid_refined.value) 
                    if not grid_refined.value:
                        solara.Markdown("Grid needs to be refined first.")
                    if plot_bathy_status.value == "Plotting":
                        solara.Markdown("Preparing plot... Please wait. This might take a couple of minutes.")
                    elif plot_bathy_status.value == "Done":
                        solara.Markdown("Done!")   
            
                    solara.Markdown("**Check Orthogonality:**")
                    solara.Button(label="Check",on_click=orthogonality_check, continuous_update=True,disabled=not grid_refined.value) 
                    if ortho_status.value == "Checking":
                        solara.Markdown("Checking Orthogonality... Please wait.")
                    elif ortho_status.value == "Done":
                        solara.Markdown("Done!")  
                        max_ortho = mk_object.mesh2d_get_orthogonality().values.max()
                        if max_ortho >= 0.4:
                            solara.Markdown(f"**Warning:** Orthogonality value is too high ({max_ortho:.2f}). Please review your grid.",
                                            style={"color": "red"})
                        else:
                            solara.Markdown(f"Orthogonality value ({max_ortho:.2f}) is below the threshold. You may proceed.")

                with solara.Row(justify="end"):     
                    solara.Button(label="Go to Step 3.", on_click=lambda: selected_tab.set('Bnd'),disabled=not gridgendone.value)


# Tab_Grid()

## 4) Boundary Conditions (tidal model and CMEMS)

In [7]:
# if _new.ext and linked files already exist:
dir_new_ext = solara.reactive("C:\\") # solara.reactive("C:\\Users\\santjer\\OneDrive - Stichting Deltares\\Documents\\IRISCC\\model")

add_ext_status = solara.reactive("Idle")
def add_wl_bnd(dir_new_ext):
    add_ext_status.set("Running")
    global ext_file_new
    ext_file_new = os.path.join(dir_new_ext.value)#, f'{model_name.value}_new.ext')
    add_ext_status.set("Done")
    bnddone.set(True) 

In [8]:
# if starting from fresh:
# Tide Model Selection -----------------------------------------------------------------------------------------------------------------------------------------
tidem_options = ["GTSMv4.1_opendap", "GTSMv4.1", "Other (not working outside Deltares)"]
tidemodel = solara.reactive("GTSMv4.1_opendap") # tidemodel: FES2014, FES2012, EOT20, GTSMv4.1, GTSMv4.1_opendap

# Generating tidal boundaries ----------------------------------------------------------------------------------------------------------------------------------
tidal_bnd_status = solara.reactive("Idle")
def tidal_bnd():
    tidal_bnd_status.set("Running")
    global ext_file_new, ext_new
    # generate new format external forcings file (.ext): initial and open boundary condition
    ext_file_new = os.path.join(dir_output.value, f'{model_name.value}_new.ext')
    ext_new = hcdfm.ExtModel()
    # interpolate tidal components to boundary conditions file (.bc)
    dfmt.interpolate_tide_to_bc(ext_new=ext_new, tidemodel=tidemodel.value, file_pli=poly_file, component_list=None)
    tidal_bnd_status.set("Completed")

# download GTSM data and restructure and make bc file ----------------------------------------------------------------------------------------------------------
download_GTSM_status = solara.reactive("Idle")
def download_GTSM():
    global dir_output_data_gtsm, ds_gtsm, ext_new
    download_GTSM_status.set("Running")
    # download GTSM data and select only grid points (no coastline data or otherwise)
    dir_output_data_gtsm = os.path.join(dir_output_data, 'GTSM')
    os.makedirs(dir_output_data_gtsm, exist_ok=True)
    gtsm_era5_gpd = dfmt.ssh_catalog_subset(source='gtsm3-era5-cds')
    subset_kwargs = dict(lon_min=lon_min.value, lon_max=lon_max.value, lat_min=lat_min.value, lat_max=lat_max.value, 
                     time_min=date_min.value.strftime("%Y-%m-%d"), time_max=date_max.value.strftime("%Y-%m-%d"))
    gtsm_era5_gpd_sel = dfmt.ssh_catalog_subset(source='gtsm3-era5-cds', **subset_kwargs)
    bool_eur = gtsm_era5_gpd_sel['station_name'].str.startswith("id_reg_grid_eur")
    gtsm_era5_gpd_sel_grid = gtsm_era5_gpd_sel.loc[bool_eur]
    #download nc files per point
    dfmt.ssh_retrieve_data(gtsm_era5_gpd_sel_grid, dir_output_data_gtsm,  
                            time_min=date_min.value.strftime("%Y-%m-%d"), time_max=date_max.value.strftime("%Y-%m-%d"))

    # restructuring
    file_pattern_gtsm = os.path.join(dir_output_data_gtsm, 'gtsm3-era5-*-id_reg_grid_eur_*.nc')
    file_list_gtsm = glob.glob(file_pattern_gtsm)
    datasets_gtsm = [xr.open_dataset(file) for file in file_list_gtsm]
    for i, ds in enumerate(datasets_gtsm):
        ds = ds.expand_dims({'stations': [ds.attrs['station_name']]})
        datasets_gtsm[i] = ds
    ds_gtsm = xr.concat(datasets_gtsm, dim='stations')

    # make bc file
    ds_gtsm = ds_gtsm.set_coords(["station_x_coordinate", "station_y_coordinate"])
    wl_GTSM = ds_gtsm[['waterlevel']]
    wl_GTSM.waterlevel.attrs['units'] = 'm'
    file_pli = Path(dir_output.value, f'{model_name.value}.pli')
    data_interp = dfmt.interp_hisnc_to_plipoints(data_xr_his=wl_GTSM, file_pli=file_pli, kdtree_k=4)
    data_interp = data_interp.rename({'waterlevel':'waterlevelbnd'}) 
    ForcingModel_object = dfmt.plipointsDataset_to_ForcingModel(plipointsDataset=data_interp)
    file_bc_out = file_pli.name.replace('.pli','.bc')
    ForcingModel_object.save(filepath=file_bc_out) #TODO REPORT: writing itself is fast, but takes quite a while to start writing (probably because of conversion)
    # Relocate bc file 
    source = os.path.join(os.getcwd(), file_bc_out)
    destination = os.path.join(dir_output.value, file_bc_out)
    shutil.move(source, destination)
    
    boundary_object = hcdfm.Boundary(quantity='waterlevelbnd', #the FM quantity for tide is also waterlevelbnd
                                        locationfile=file_pli,
                                        forcingfile=ForcingModel_object)
    dfmt.interpolate_grid2bnd.ext_add_boundary_object_per_polyline(ext_new=ext_new, boundary_object=boundary_object)
    ext_new.save(filepath=ext_file_new)

    download_GTSM_status.set("Completed")


# additional CMEMS xuxyadvectionvelocity boundary
# CMEMS boundaries ---------------------------------------------------------------------------------------------------------------------------------------------
cmems_bnd_status = solara.reactive("Idle")
def cmems_bnd():
    global ext_new
    cmems_bnd_status.set("Running")
    dir_output_data_cmems = os.path.join(dir_output_data, 'cmems')
    os.makedirs(dir_output_data_cmems, exist_ok=True)
    for varkey in ['uo','vo']:
        dfmt.download_CMEMS(varkey=varkey,
                            longitude_min=lon_min.value-1, longitude_max=lon_max.value+1, latitude_min=lat_min.value-1, latitude_max=lat_max.value+1,
                            date_min=date_min.value.strftime("%Y-%m-%d"), date_max=date_max.value.strftime("%Y-%m-%d"),
                            dir_output=dir_output_data_cmems, file_prefix='cmems_', overwrite=overwrite)        

    list_quantities = ['uxuyadvectionvelocitybnd']
    dir_pattern = os.path.join(dir_output_data_cmems,'cmems_{ncvarname}_*.nc')
    ref_date = date_min
    ext_new = dfmt.cmems_nc_to_bc(ext_new=ext_new,
                                refdate_str=f'minutes since {ref_date.value.strftime("%Y-%m-%d")} 00:00:00 +00:00',
                                dir_output=dir_output.value,
                                list_quantities=list_quantities,
                                tstart=date_min.value.strftime("%Y-%m-%d"),
                                tstop=date_max.value.strftime("%Y-%m-%d"), 
                                file_pli=poly_file,
                                dir_pattern=dir_pattern)
    ext_new.save(filepath=ext_file_new)
    cmems_bnd_status.set("Completed")
    bnddone.set(True) 



In [9]:
@solara.component
def Tab_Boundary_Cond():

    with solara.lab.Tabs(): 
            with solara.lab.Tab("Import Existing"):
                with solara.Card("Boundary Conditions", style={"width": "100%", "padding": "10px"}):
                    solara.Markdown("Follow this in case an external forcing file ('_new.ext') exists including ALL linked necessary boundary files (i.e. '.bc' and '.pli') for the model case.")
    
                    solara.Markdown("**External Forcing file:**")
                    solara.InputText("Directory of the external forcing file ('_new.ext')", value=dir_new_ext, continuous_update=True)
                    # if dir_new_ext.value and not dir_new_ext.value.endswith("_new.ext"):
                    #     solara.Error(label='Error: File must end with "_new.ext"', text=False, dense=True, outlined=True, icon=False)
                    path_new_ext = Path(dir_new_ext.value); newext_valid = False
                    if dir_new_ext.value:
                        if not dir_new_ext.value.endswith("_new.ext"):
                            solara.Error(label='Error: File must end with "_new.ext"', text=False, dense=True, outlined=True, icon=False)
                        elif not path_new_ext.exists():
                            solara.Error("File does not exist.")
                        else:
                            newext_valid = True

                    
                    solara.Button(label="Add File",on_click=lambda: add_wl_bnd(dir_new_ext), continuous_update=True, disabled=not newext_valid)
                    if add_ext_status.value == "Done":
                        solara.Markdown(f"External Forcing file added: '{ext_file_new}'")
                with solara.Row(justify="end"):     
                    solara.Button(label="Go to Step 4.", on_click=lambda: selected_tab.set('Forcing'),disabled=not bnddone.value)




            with solara.lab.Tab("Create New"):
                with solara.Card("Boundary Conditions", style={"width": "100%", "padding": "10px"}):
                    solara.Markdown("**Tide Model:**")
                    solara.Select(label="Tide Model", value=tidemodel, values=tidem_options)
            
                    solara.Button(label="Create Tidal Boundaries",on_click=lambda: (tidal_bnd(), tidalbnddone.set(True)), continuous_update=True)
                    if tidal_bnd_status.value == "Running":
                        solara.Markdown("**Creating...** Please wait. This takes a couple of minutes.")
                    elif tidal_bnd_status.value == "Completed":
                        solara.Markdown("**Tidal Boundaries created successfully!**") 
                        
                    solara.Markdown("**Download GTSM data:**")
                    solara.Markdown("Note: Download of GTSM data only implemented for European areas.")
                    solara.Button(label="Download",on_click=lambda: (download_GTSM(), downloadGTSMdone.set(True)), continuous_update=True, disabled=not tidalbnddone.value) # solara.Button(label="Download",on_click=download_GTSM, continuous_update=True)
                    if not tidalbnddone.value:
                        solara.Markdown("Tidal Boundaries need to be generated first.")
                    if download_GTSM_status.value == "Running":
                        solara.Markdown("**Downloading...** Please wait. This takes a couple of minutes.")
                    elif download_GTSM_status.value == "Completed":
                        solara.Markdown("**GTSM data downloaded successfully!**")
                    
                    solara.Markdown("**Get data from CMEMS:**")
                    solara.Button(label="Download",on_click=cmems_bnd, continuous_update=True, disabled=not downloadGTSMdone.value)
                    if not downloadGTSMdone.value:
                        solara.Markdown("GTSM data needs to be downloaded first.")
                    if cmems_bnd_status.value == "Running":
                        solara.Markdown("**Downloading...** Please wait. This might take a couple of minutes.")
                    elif cmems_bnd_status.value == "Completed":
                        solara.Markdown("**Data downloaded and saved successfully!**") 
                with solara.Row(justify="end"):     
                    solara.Button(label="Go to Step 4.", on_click=lambda: selected_tab.set('Forcing'),disabled=not bnddone.value)

# Tab_Boundary_Cond()

## 5) Generate CMEMS ini cond and ERA5 meteo forcing

In [10]:
# if _old.ext file (and those referred to within) already exist:
dir_old_ext = solara.reactive("C:\\") # solara.reactive("C:\\Users\\santjer\\OneDrive - Stichting Deltares\\Documents\\IRISCC\\model")

add_old_ext_status = solara.reactive("Idle")
def add_old_bnd(dir_old_ext):
    add_old_ext_status.set("Running")
    global ext_file_old
    ext_file_old = os.path.join(dir_old_ext.value)#, f'{model_name.value}_old.ext') # HIER HIER HIER vorher _new_
    add_old_ext_status.set("Done")
    forcingdone.set(True)

In [11]:
# if starting from fresh:
gen_ext_forcing_status = solara.reactive("Idle")
def gen_ext_forcing():
    global ext_file_old, ext_old
    gen_ext_forcing_status.set("Running")
    # generate old format external forcings file (.ext): spatial data
    ext_file_old = os.path.join(dir_output.value, f'{model_name.value}_old.ext')
    ext_old = hcdfm.ExtOldModel()
    gen_ext_forcing_status.set("Completed")


download_save_ERA5_status = solara.reactive("Idle")
def download_ERA5():
    global varlist_list, dir_output_data_era5, ext_old
    download_save_ERA5_status.set("Running")
    # ERA5 - download spatial fields of air pressure, wind speeds and Charnock coefficient
    dir_output_data_era5 = os.path.join(dir_output_data, 'ERA5')
    os.makedirs(dir_output_data_era5, exist_ok=True)
        
    varlist_list = [['msl','u10n','v10n','chnk']]
    for varlist in varlist_list:
        for varkey in varlist:
            dfmt.download_ERA5(varkey, 
                            longitude_min=lon_min.value, longitude_max=lon_max.value, latitude_min=lat_min.value, latitude_max=lat_max.value,
                            date_min=date_min.value.strftime("%Y-%m-%d"), date_max=date_max.value.strftime("%Y-%m-%d"),
                            dir_output=dir_output_data_era5, overwrite=overwrite)
    download_save_ERA5_status.set("Completed")
    
    # ERA5 meteo - convert to netCDF for usage in Delft3D FM
    ext_old = dfmt.preprocess_merge_meteofiles_era5(ext_old=ext_old,
                                                    varkey_list=['msl','u10n','v10n','chnk'],# varlist_list,
                                                    dir_data=dir_output_data_era5,
                                                    dir_output=dir_output.value,
                                                    time_slice=slice(date_min.value.strftime("%Y-%m-%d"), date_max.value.strftime("%Y-%m-%d")))

    ext_old.save(filepath=ext_file_old) # , path_style=path_style)
    forcingdone.set(True)

In [12]:
@solara.component
def Tab_Forcings():

    with solara.lab.Tabs(): 
            with solara.lab.Tab("Import Existing"):
                with solara.Card("Forcings", style={"width": "100%", "padding": "10px"}):
                    solara.Markdown("Follow this in case an external forcing file ('_old.ext') exists including ALL linked necessary files for the model case.")
    
                    solara.Markdown("**External Forcing file:**")
                    solara.InputText("Directory of the external forcing file ('_old.ext')", value=dir_old_ext, continuous_update=True)
                    path_old_ext = Path(dir_old_ext.value); oldext_valid = False
                    if dir_old_ext.value:
                        if not dir_old_ext.value.endswith("_old.ext"):
                            solara.Error(label='Error: File must end with "_old.ext"', text=False, dense=True, outlined=True, icon=False)
                        elif not path_old_ext.exists():
                            solara.Error("File does not exist.")
                        else:
                            oldext_valid = True

                    solara.Button(label="Add File",on_click=lambda: add_old_bnd(dir_old_ext), continuous_update=True, disabled=not oldext_valid)
                    if add_old_ext_status.value == "Done":
                        solara.Markdown(f"External Forcing file added: '{ext_file_old}'")

                with solara.Row(justify="end"):     
                    solara.Button(label="Go to Step 5.", on_click=lambda: selected_tab.set('Obs'),disabled=not forcingdone.value)


            with solara.lab.Tab("Create New"):
    
                with solara.Card("Forcings", style={"width": "100%", "padding": "10px"}):
                    solara.Markdown("**Generate external forcing file:**")
                    solara.Button(label="Generate",on_click=lambda: (gen_ext_forcing(), genextforcingdone.set(True)), continuous_update=True)
                    if gen_ext_forcing_status.value == "Running":
                        solara.Markdown("**Generating ...** Please wait. This might take a couple of minutes.")
                    elif gen_ext_forcing_status.value == "Completed":
                        solara.Markdown("**Generation successfully!**")
            
                    solara.Markdown("**Download and save ERA5 data:**")
                    solara.Button(label="Download & Save",on_click=download_ERA5, continuous_update=True, disabled=not genextforcingdone.value)
                    if not genextforcingdone.value:
                        solara.Markdown("Generating external forcing file needs to be done first.")
                    if download_save_ERA5_status.value == "Running":
                        solara.Markdown("**Download and Save ...** Please wait. This might take a couple of minutes.")
                    elif download_save_ERA5_status.value == "Completed":
                        solara.Markdown("**Download & Save successful!**") 

                with solara.Row(justify="end"):     
                    solara.Button(label="Go to Step 5.", on_click=lambda: selected_tab.set('Obs'),disabled=not forcingdone.value)

# Tab_Forcings()

## 6) Generate obsfile

- either input box (float) for observation location (x, y)
- draw in map (not yet implemented)

both options:
- name per selected location (x,y pair) ------>>> x, y & name as DataFrame!
- to be added to a list and all shown in map somehow

In [13]:
# if .xyn file already exists:

dir_xyn = solara.reactive("C:\\") # solara.reactive("C:\\Users\\santjer\\OneDrive - Stichting Deltares\\Documents\\IRISCC\\model") # 

add_xyn_status = solara.reactive("Idle")
def add_xyn(dir_xyn, size=0.1):
    add_xyn_status.set("Running")
    global file_obs
    file_obs = os.path.join(dir_xyn.value)#, f'{model_name.value}_obs.xyn')


    # add on map ----------------------------------------------------
    if first_run_flag.value:
        first_run_flag.set(False)
        for layer in list(current_layer_group.value.layers):
            current_layer_group.value.remove_layer(layer)


    data = np.loadtxt(file_obs, usecols=(0, 1)); data = np.atleast_2d(data) #######NEU NEU NEU NEU NEU
    x = data[:, 0]; y = data[:, 1]
    
    for lat, lon in zip(y, x):
        vertical = Polyline(locations=[(lat - size, lon), (lat + size, lon)],color="red", weight=2)
        horizontal = Polyline(locations=[(lat, lon - size), (lat, lon + size)],color="red", weight=2)
        
        current_layer_group.value.add_layer(horizontal)
        current_layer_group.value.add_layer(vertical)
    
    add_xyn_status.set("Done")
    obsdone.set(True)

In [14]:
# if starting from fresh:
obs_name = solara.reactive("Location_Name")

first_run_flag = solara.reactive(True)
def add_cross(lon, lat, size=0.1): # add on map ----------------------------------------------------
    if first_run_flag.value:
        first_run_flag.set(False)
        for layer in list(current_layer_group.value.layers):
            current_layer_group.value.remove_layer(layer)
            
    vertical = Polyline(
        locations=[(lat - size, lon), (lat + size, lon)],
        color="red", weight=2)
    horizontal = Polyline(
        locations=[(lat, lon - size), (lat, lon + size)],
        color="red", weight=2)

    current_layer_group.value.add_layer(horizontal) 
    current_layer_group.value.add_layer(vertical) 


xi = solara.reactive(0.0) # lon!
yi = solara.reactive(0.0) # lat!

obs_list = []; obs_name_list = []
formatted_locations = solara.reactive("") 
def add_loc(): # add loc given textbox inputs -------------------------------------------------------
    global obs_list, obs_name_list
    obs_list.append((xi.value, yi.value)) # (lon, lat)
    obs_name_list.append((obs_name.value))
    formatted_locations.set("; ".join(f"({round(lon, 1)}, {round(lat, 1)})" for lon, lat in obs_list))
    
    add_cross(xi.value, yi.value) # x, y --> lon, lat

    obs_name.set("Location_Name")
    xi.set(0.0) # lon!            
    yi.set(0.0) # lat!            

def clear_obs(): # clear all obs from textbox inputs -------------------------------------------------
    global obs_list
    obs_list = []
    formatted_locations.set("")
    # remove layers from map:
    layer_group = tab_layers[selected_tab.value]
    layers_to_remove = [layer for layer in layer_group.layers if isinstance(layer, Polyline) and layer.color == 'red']
    for layer in layers_to_remove:
        layer_group.remove_layer(layer)
    for layer in list(current_layer_group.value.layers):
        current_layer_group.value.remove_layer(layer)

def save_obs():
    global obs_pd, file_obs
    obs_pd = pd.DataFrame(zip(*zip(*obs_list), obs_name_list), columns=["name","x","y"])

    file_obs = os.path.join(dir_output.value, f'{model_name.value}_obs.xyn')
    obs_pd.to_csv(file_obs, sep=' ', header=False, index=False, float_format='%.6f')
    
    obsdone.set(True)
        

In [15]:
@solara.component
def Tab_Observation():
    with solara.lab.Tabs(): 
        with solara.lab.Tab("Import Existing"):
            with solara.Card("Observations", style={"width": "100%", "padding": "10px"}):
                solara.Markdown("Follow this in case an observation file ('_obs.xyn') exists for the model case.")
                #
                solara.Markdown("**Observation file:**")
                solara.InputText("Directory of the observation file ('_obs.xyn')", value=dir_xyn, continuous_update=True)
                # if dir_xyn.value and not dir_xyn.value.endswith("_obs.xyn"):
                #     solara.Error(label='Error: File must end with "_obs.xyn"', text=False, dense=True, outlined=True, icon=False)
                path_xyn = Path(dir_xyn.value); obsxyn_valid = False
                if dir_xyn.value:
                    if not dir_xyn.value.endswith("_obs.xyn"):
                        solara.Error(label='Error: File must end with "_obs.xyn"', text=False, dense=True, outlined=True, icon=False)
                    elif not path_xyn.exists():
                        solara.Error("File does not exist.")
                    else:
                        obsxyn_valid = True

                solara.Button(label="Add File",on_click=lambda: add_xyn(dir_xyn), continuous_update=True, disabled=not obsxyn_valid)
                if add_xyn_status.value == "Done":
                    solara.Markdown(f"Observation file added: '{file_obs}'")

            with solara.Row(justify="end"):     
                solara.Button(label="Go to Step 6.", on_click=lambda: selected_tab.set('mdu'),disabled=not obsdone.value)


        with solara.lab.Tab("Create New"):    
            with solara.Card("Observations", style={"width": "100%", "padding": "10px"}):
                solara.Markdown("Create new Observation Location:")
                solara.InputText("Observation Location Name", value=obs_name, continuous_update=True)
                solara.InputFloat("Latitude", value=yi, continuous_update=True)
                solara.InputFloat("Longitude", value=xi, continuous_update=True)
                solara.Markdown(" . ") # to get space between input and button
                solara.Button("Add location", on_click=add_loc) 
                solara.Markdown(f"**Locations:** {formatted_locations.value}")
        
                solara.Button("Clear Locations", on_click=clear_obs)
        
                solara.Button("Save Locations", on_click=save_obs)

            with solara.Row(justify="end"):     
                solara.Button(label="Go to Step 6.", on_click=lambda: selected_tab.set('mdu'),disabled=not obsdone.value)

# Tab_Observation()

## 7) Generate mdu file

In [16]:
# settings/choices open for user
hisint = solara.reactive(600)
mapint = solara.reactive(1800)
rstint = solara.reactive(0)
statsint = solara.reactive(3600)

dimrset_folder = solara.reactive(r"p:\d-hydro\dimrset\weekly\2.28.04") 

In [17]:
# # if .mdu and other files already exists:

# gen_mdu_dimr_status_ifexist = solara.reactive("Idle")
# def generate_mdu():
#     global mdu_file
#     gen_mdu_dimr_status_ifexist.set("Running")
#     # initialize mdu file and update settings
#     mdu_file = os.path.join(dir_output.value, 'test.mdu')
#     mdu = hcdfm.FMModel()

#     # add the grid (_net.nc, network file)
#     mdu.geometry.netfile = netfile

#     # create and add drypointsfile if there are any cells generated that will result in high orthogonality
#     try: # HIER JA, WENN ILLEGALCELLS_GDF EXIST! sprich: wenn grid teil neu gemacht wurde
#         illegalcells_gdf 
#         if len(illegalcells_gdf) > 0:
#             illegalcells_polyfile = dfmt.geodataframe_to_PolyFile(illegalcells_gdf)
#             illegalcells_file = os.path.join(dir_output.value, "illegalcells.pol")
#             illegalcells_polyfile.save(illegalcells_file)
#             mdu.geometry.drypointsfile = [illegalcells_polyfile]
#     except NameError: # ELSE: überspringen?!!!!!
#         print('')

#     # add the external forcing files (.ext)
#     mdu.external_forcing.extforcefile = ext_file_old
#     mdu.external_forcing.extforcefilenew = ext_file_new

#     mdu.geometry.openboundarytolerance = 0.1
#     mdu.geometry.bedlevuni = 5 # default of -5 may cause instabilities at coastline
#     mdu.geometry.dxwuimin2d = 0.1 # improved stability in triangular network cells
#     # update numerics settings
#     mdu.numerics.izbndpos = 1 # boundary points are on network boundary
#     mdu.numerics.mintimestepbreak = 0.1 # causes instable model to crash
#     mdu.numerics.keepstbndonoutflow = 1 # s/t on outflow boundaries, should be new default
#     mdu.numerics.barocponbnd = 1 # enable baroclinic pressure gradient on open boundaries, should be new default
#     mdu.numerics.vertadvtypsal = 4 # should be new default (is currently 6)
#     mdu.numerics.vertadvtyptem = 4 # should be new default (is currently 6)
#     # update physics settings
#     mdu.physics.rhomean = 1023. # for coastal models
#     # update wind settings
#     mdu.wind.icdtyp = 4 # Charnock in case of ERA5
#     mdu.wind.cdbreakpoints = [0.025] # important if icdtyp=4, but 0.018 or 0.041 might be better. Value is overwritten by spacevarying charnock from ERA5.
#     mdu.wind.pavbnd = 101330 # for inverse barometer correction, important in case of CMEMS boundary conditions
#     # update time settings
#     mdu.time.refdate = pd.Timestamp(date_min.value.strftime("%Y-%m-%d")).strftime('%Y%m%d') # pd.Timestamp(ref_date.value.strftime("%Y-%m-%d")).strftime('%Y%m%d')
#     mdu.time.tunit = 'S'
#     mdu.time.dtmax = 30
#     mdu.time.startdatetime = pd.Timestamp(date_min.value.strftime("%Y-%m-%d")).strftime('%Y%m%d%H%M%S')
#     mdu.time.stopdatetime = pd.Timestamp(date_max.value.strftime("%Y-%m-%d")).strftime('%Y%m%d%H%M%S')
    
#     mdu.output.obsfile = [file_obs]
#     mdu.output.hisinterval = [hisint.value]
#     mdu.output.mapinterval = [mapint.value]#[86400]
#     mdu.output.rstinterval = [rstint.value]
#     mdu.output.statsinterval = [statsint.value]
    
    
#     mdu.save(mdu_file)
    
#     dfmt.make_paths_relative(mdu_file)


#     # CHAPTER 8 CODE!!! -----------------------------------------------------------------------------------------------------
#     nproc = 1 # number of processes
#     # dimrset_folder = None # previously r"p:\d-hydro\dimrset\weekly\2.28.04" # alternatively r"c:\Program Files\Deltares\Delft3D FM Suite 2024.03 HMWQ\plugins\DeltaShell.Dimr\kernels" #alternatively r"p:\d-hydro\dimrset\weekly\2.28.04"
#     dfmt.create_model_exec_files(file_mdu=mdu_file, nproc=nproc, dimrset_folder=dimrset_folder.value)
#     # remove pause at end of bat file
#     bat_loc = os.path.join(dir_output.value, 'run_parallel.bat')
#     with open(bat_loc, 'r') as file:
#         lines = file.readlines()
#     if len(lines) > 1 and 'pause' in lines[-1]:
#         lines.pop(-1)
#     with open(bat_loc, 'w') as file:
#         file.writelines(lines)
        
#     gen_mdu_dimr_status_ifexist.set("Completed")

In [18]:
# if starting from fresh:

gen_mdu_dimr_status = solara.reactive("Idle")
def generate_mdu():
    global mdu_file
    gen_mdu_dimr_status.set("Running")
    # initialize mdu file and update settings
    mdu_file = os.path.join(dir_output.value, f'{model_name.value}.mdu')
    mdu = hcdfm.FMModel()

    # add the grid (_net.nc, network file)
    mdu.geometry.netfile = netfile
    mdu.geometry.openboundarytolerance = 0.1
    mdu.geometry.bedlevuni = 5 # default of -5 may cause instabilities at coastline
    mdu.geometry.dxwuimin2d = 0.1 # improved stability in triangular network cells

    # create and add drypointsfile if there are any cells generated that will result in high orthogonality
    try: # if ILLEGALCELLS_GDF EXIST!
        illegalcells_gdf 
        if len(illegalcells_gdf) > 0:
            illegalcells_polyfile = dfmt.geodataframe_to_PolyFile(illegalcells_gdf)
            illegalcells_file = os.path.join(dir_output.value, "illegalcells.pol")
            illegalcells_polyfile.save(illegalcells_file)
            mdu.geometry.drypointsfile = [illegalcells_polyfile]
    except NameError: # else do nothing here
        print('')

    # update numerics settings
    mdu.numerics.izbndpos = 1 # boundary points are on network boundary
    mdu.numerics.mintimestepbreak = 0.1 # causes instable model to crash
    mdu.numerics.keepstbndonoutflow = 1 # s/t on outflow boundaries, should be new default
    mdu.numerics.barocponbnd = 1 # enable baroclinic pressure gradient on open boundaries, should be new default
    mdu.numerics.vertadvtypsal = 4 # should be new default (is currently 6)
    mdu.numerics.vertadvtyptem = 4 # should be new default (is currently 6)

    # update physics settings
    mdu.physics.rhomean = 1023. # for coastal models

    # update wind settings
    mdu.wind.icdtyp = 4 # Charnock in case of ERA5
    mdu.wind.cdbreakpoints = [0.025] # important if icdtyp=4, but 0.018 or 0.041 might be better. Value is overwritten by spacevarying charnock from ERA5.
    mdu.wind.pavbnd = 101330 # for inverse barometer correction, important in case of CMEMS boundary conditions


    # update time settings
    mdu.time.refdate = pd.Timestamp(ref_date.value.strftime("%Y-%m-%d")).strftime('%Y%m%d')
    mdu.time.tunit = 'S'
    mdu.time.dtmax = 30
    mdu.time.startdatetime = pd.Timestamp(date_min.value.strftime("%Y-%m-%d")).strftime('%Y%m%d%H%M%S')
    mdu.time.stopdatetime = pd.Timestamp(date_max.value.strftime("%Y-%m-%d")).strftime('%Y%m%d%H%M%S')

    # add the external forcing files (.ext)
    mdu.external_forcing.extforcefile = ext_file_old
    # mdu.external_forcing.extforcefilenew = ext_new
    try:
        mdu.external_forcing.extforcefilenew = ext_new # eigentlich ext_new; ABER: when importing existing files, ext_new is not defined
    except NameError: # so when importing existing files, follow this (leads to same result in the mdu file)
        mdu.external_forcing.extforcefilenew = ext_file_new #

    mdu.output.obsfile = [file_obs]
    mdu.output.hisinterval = [hisint.value]
    mdu.output.mapinterval = [mapint.value]#[86400]
    mdu.output.rstinterval = [rstint.value]
    mdu.output.statsinterval = [statsint.value]

    # save .mdu file
    mdu.save(mdu_file)

    # make all paths relative (might be properly implemented in https://github.com/Deltares/HYDROLIB-core/issues/532)
    dfmt.make_paths_relative(mdu_file)

    # CHAPTER 8 CODE!!! -----------------------------------------------------------------------------------------------------
    nproc = 1 # number of processes
    # dimrset_folder = None # previously r"p:\d-hydro\dimrset\weekly\2.28.04" # alternatively r"c:\Program Files\Deltares\Delft3D FM Suite 2024.03 HMWQ\plugins\DeltaShell.Dimr\kernels" #alternatively r"p:\d-hydro\dimrset\weekly\2.28.04"
    dfmt.create_model_exec_files(file_mdu=mdu_file, nproc=nproc, dimrset_folder=dimrset_folder.value)
    # remove pause at end of bat file
    bat_loc = os.path.join(dir_output.value, 'run_parallel.bat')
    with open(bat_loc, 'r') as file:
        lines = file.readlines()
    if len(lines) > 1 and 'pause' in lines[-1]:
        lines.pop(-1)
    with open(bat_loc, 'w') as file:
        file.writelines(lines)
        
    gen_mdu_dimr_status.set("Completed")
    mdudone.set(True)



In [19]:
@solara.component
def Tab_gen_mdu():

    with solara.Card("Generate mdu, DIMR and bat files", style={"width": "100%", "padding": "10px"}):

        solara.Text("his time interval:")
        solara.InputFloat("his Interval", value=hisint, continuous_update=True)
        solara.Text("map time interval:")
        solara.InputFloat("map Interval", value=mapint, continuous_update=True)
        solara.Text("restart time interval:")
        solara.InputFloat("rst Interval", value=rstint, continuous_update=True)
        solara.Text("Stats time interval:")
        solara.InputFloat("stats Interval", value=statsint, continuous_update=True)

        solara.Text(".")
        solara.Markdown("""**DIMR and bat file generation**: In order to run the model via DIMR we need a dimr_config.xml file. 
                    If you are running this notebook on a Windows platform, a *.bat file 
                    will also be created with which you can run the model directly. 
                    In order for this to work you need to update the dimrset_folder to the
                    path where the x64 and or lnx64 folder is located. Provide None if you
                    have no D-Flow FM executable available on your system.""")
        solara.InputText("DIMR location", value=dimrset_folder, continuous_update=True)
        
        solara.Button("Initialise files",on_click=generate_mdu)
        if gen_mdu_dimr_status.value == "Running":
            solara.Markdown("**Initialising ...** Please wait.")
        elif gen_mdu_dimr_status.value == "Completed":
            solara.Markdown("**Initialisation successfully!**")

    with solara.Row(justify="end"):     
        solara.Button(label="Go to Step 7.", on_click=lambda: selected_tab.set('run'),disabled=not mdudone.value)


# Tab_gen_mdu()
       

## 8) Generate DIMR and bat file

In same tab as 7) (generate files)

## 9) Visualise model tree

not necessary for now, difficult to show in widget

In [20]:
# mdu_obj = hcdfm.FMModel(mdu_file)
# mdu_obj.show_tree()

## 10) Run Model

In [21]:
def check_file_size(file_path, check_interval=0.5, stabilization_checks=5):
    while not os.path.exists(file_path):
        time.sleep(check_interval)
    previous_size = -1
    stable_count = 0
    while True:
        current_size = os.path.getsize(file_path)
        if current_size == previous_size:
            stable_count += 1
            if stable_count >= stabilization_checks:
                break
        else:
            stable_count = 0
        previous_size = current_size
        time.sleep(check_interval)


model_run_status = solara.reactive("Idle")

def run_model():
    model_run_status.set("Running")
    global file_nc_his, file_nc_map

    # execute bat
    bat_loc = os.path.join(dir_output.value, 'run_parallel.bat')
    subprocess.Popen(bat_loc, shell=True, cwd=dir_output.value)
    
    file_nc_his = os.path.join(dir_output.value, f"DFM_OUTPUT_{model_name.value}", f"{model_name.value}_his.nc")
    file_nc_map = os.path.join(dir_output.value, f"DFM_OUTPUT_{model_name.value}", f"{model_name.value}_map.nc")
    check_file_size(file_nc_map)
    model_run_status.set("Completed")
    rundone.set(True)

In [22]:
@solara.component
def Tab_run_model():
    
    with solara.Card("Run Model", style={"width": "100%", "padding": "10px"}):
      
        solara.Button("Run Model",on_click=run_model)
        if model_run_status.value == "Running":
            solara.Markdown("**Running ...** Please wait. This might take some time, depending on model domain.")
        elif model_run_status.value == "Completed":
            solara.Markdown("**Model run successful!**")

# Tab_run_model()     

## 11) a) Visualisation Timeseries

In [23]:
layer = None
raster_res = 0.5
umag_clim = None
scale = 15

try:
    file_nc_his
except NameError:
    file_nc_his = None
file_nc_his_new = solara.reactive("")   # user-specified path
use_manual_nc = solara.reactive(False)  # switch state
plot_fig = solara.reactive(None)

def print_timeseries():
    nc_file_to_use = None    
    if file_nc_his is not None and not use_manual_nc.value:
        nc_file_to_use = file_nc_his
    elif file_nc_his_new.value.strip() != "":
        nc_file_to_use = file_nc_his_new.value.strip()

    if not nc_file_to_use or not os.path.exists(nc_file_to_use):
        print("no netcdf file provided")
        return

    try:
        ds_his = xr.open_dataset(nc_file_to_use)
    except (TypeError, ValueError):
        ds_his = xr.open_mfdataset(nc_file_to_use)

    fig = Figure(figsize=(5, 2.5))
    ax = fig.subplots(1, 1)
    ds_his.waterlevel.plot.line(ax=ax, x='time')
    ax.legend(ds_his.station.to_series(), loc=1, fontsize=8)
    ax.grid()

    plot_fig.set(fig)



In [24]:
@solara.component
def Tab_visu1():
  
    solara.use_effect(lambda: plot_fig.set(None), [use_manual_nc.value])
    
    with solara.Card("Timeseries", style={"width": "100%", "padding": "10px"}):

        # if not rundone.value or file_nc_his is None:
        solara.Switch(label="Manual file path", value=use_manual_nc)
        if use_manual_nc.value:
            solara.InputText("Path to .nc file", value=file_nc_his_new, continuous_update=True)
            path_file_nc_his_new = Path(file_nc_his_new.value)
            if file_nc_his_new.value:
                if not file_nc_his_new.value.endswith("_his.nc"):
                    solara.Error(label='Error: File must end with "_his.nc"',text=False,dense=True,outlined=True,icon=False)
                elif not path_file_nc_his_new.exists():
                    solara.Error("File does not exist.")

        
        solara.Button("Create Figure",on_click=print_timeseries)
        if plot_fig.value is not None:
            solara.FigureMatplotlib(plot_fig.value)

# Tab_visu1()

## 11) b) Visualisation Water levels

In [25]:
# WITHIN MAP m

try:
    file_nc_map
except NameError:
    file_nc_map = None
file_nc_map_new = solara.reactive("")      # user input path
use_manual_map = solara.reactive(False)    # switch state
plot_wl_map_status = solara.reactive("Idle")
# legend_wl = []
def print_wl_map():
    plot_wl_map_status.set("Plotting")
    global uds_map # legend_wl    

    nc_file_to_use = None
    if file_nc_map is not None and not use_manual_map.value:
        nc_file_to_use = file_nc_map
    elif file_nc_map_new.value.strip():
        nc_file_to_use = file_nc_map_new.value.strip()

    if not nc_file_to_use or not os.path.exists(nc_file_to_use):
        plot_wl_map_status.set("Invalid path")
        print(f"Invalid or missing NetCDF map file: {nc_file_to_use}")
        return
    
    uds_map = dfmt.open_partitioned_dataset(nc_file_to_use)#file_nc_map)
    bool_drycells = uds_map['mesh2d_s1']==uds_map['mesh2d_flowelem_bl']
    uds_map['mesh2d_s1_filt'] = uds_map['mesh2d_s1'].where(~bool_drycells)
    gdf_wl = uds_map['mesh2d_s1_filt'].isel(time=3).ugrid.to_geodataframe(name="waterlevel")
    # remove NaNs from gdf_wl
    gdf_wl_cleaned = gdf_wl.dropna(subset=["waterlevel"])
    min_wl, max_wl = gdf_wl_cleaned["waterlevel"].min(), gdf_wl_cleaned["waterlevel"].max()
    colormap = linear.viridis.scale(round(min_wl,2), round(max_wl,2))
    sorted_wl = sorted(gdf_wl_cleaned["waterlevel"])
    
    scatter_layer = LayerGroup()
    for _, row in gdf_wl_cleaned.iterrows():
        color = colormap((row["waterlevel"]))  # Map value to color
        marker = CircleMarker(
            location=(row["mesh2d_face_y"], row["mesh2d_face_x"]),
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        )
        scatter_layer.add_layer(marker)
        
    plot_wl_map_status.set("Done")
    for layer in list(current_layer_group.value.layers):
        current_layer_group.value.remove_layer(layer)
    if rectangle in current_layer_group.value.layers: 
        current_layer_group.value.remove_layer(rectangle) 
    current_layer_group.value.add_layer(scatter_layer)
    
    # LEGEND/COLORBAR
    legend_html = widgets.HTML(value=colormap._repr_html_())
    legend_control = WidgetControl(widget=legend_html, position='bottomright')
    tab_controls.value["visu2"] = legend_control
    control_update_signal.set(control_update_signal.value + 1)

        

In [26]:
@solara.component
def Tab_visu2():
    solara.use_effect(lambda: plot_wl_map_status.set("Idle"), [use_manual_map.value])
    
    with solara.Card("Water levels [m]", style={"width": "100%", "padding": "10px"}):
        
        solara.Switch(label="Manual map file path", value=use_manual_map)
        if use_manual_map.value:
            solara.InputText(label="Path to .nc map file", value=file_nc_map_new, continuous_update=True)
            path_file_nc_map_new = Path(file_nc_map_new.value)
            if file_nc_map_new.value:
                if not file_nc_map_new.value.endswith("_map.nc"):
                    solara.Error(label='Error: File must end with "_map.nc"',text=False,dense=True,outlined=True,icon=False)
                elif not path_file_nc_map_new.exists():
                    solara.Error("File does not exist.")
        
        solara.Button("Create Map",on_click=print_wl_map)
        if plot_wl_map_status.value == "Plotting":
            solara.Markdown("Preparing plot... Please wait. This might take a couple of minutes.")
        elif plot_wl_map_status.value == "Done":
            solara.Markdown("**Done!**")
            
# Tab_visu2()

## 11) c) Visualisation Currents

In [27]:
try:
    file_nc_map
except NameError:
    file_nc_map = None
file_nc_map_new = solara.reactive("")      # user input path
use_manual_map = solara.reactive(False)    # switch state
plot_curr_map_status = solara.reactive("Idle")
legend_currents = []
def print_current_map():
    global legend_currents
    plot_curr_map_status.set("Plotting")

    nc_file_to_use = None
    if file_nc_map is not None and not use_manual_map.value:
        nc_file_to_use = file_nc_map
    elif file_nc_map_new.value.strip():
        nc_file_to_use = file_nc_map_new.value.strip()

    if not nc_file_to_use or not os.path.exists(nc_file_to_use):
        plot_wl_map_status.set("Invalid path")
        print(f"Invalid or missing NetCDF map file: {nc_file_to_use}")
        return
    
    scaling_factor = 0.15
    uds_map = dfmt.open_partitioned_dataset(nc_file_to_use)#file_nc_map)
    uds_quiv = uds_map.isel(time=-1, mesh2d_nLayers=-2, nmesh2d_layer=-2, missing_dims='ignore')
    magn_attrs = {'long_name':'velocity magnitude', 'units':'m/s'}
    varn_ucx, varn_ucy = 'mesh2d_ucx', 'mesh2d_ucy'
    uds_quiv['magn'] = np.sqrt(uds_quiv[varn_ucx]**2+uds_quiv[varn_ucy]**2).assign_attrs(magn_attrs)
    raster_quiv = dfmt.rasterize_ugrid(uds_quiv[[varn_ucx,varn_ucy]], resolution=raster_res)

    # COLORMAP/SCATTER
    mag = uds_quiv['magn'].values
    x = uds_quiv['magn']['mesh2d_face_x'].values # mesh2d_node_x
    y = uds_quiv['magn']['mesh2d_face_y'].values # mesh2d_node_y
    min_mag = min(mag)
    max_mag = max(mag)
    colormap = linear.viridis.scale(round(min_mag,2), round(max_mag,2))
    
    scatter_layer = LayerGroup()
    for i in range(len(mag)):
        color = colormap(mag[i])  # Map value to color
        marker = CircleMarker(
            location=(y[i], x[i]),
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        )
        scatter_layer.add_layer(marker)

    # LEGEND/COLORBAR 
    legend_html = widgets.HTML(value=colormap._repr_html_())
    legend_control = WidgetControl(widget=legend_html, position='bottomright')
    tab_controls.value["visu3"] = legend_control 
    control_update_signal.set(control_update_signal.value + 1) 
    
    # QUIVER
    x = raster_quiv.mesh2d_face_x.values
    y = raster_quiv.mesh2d_face_y.values
    u = raster_quiv.mesh2d_ucx.values # varn_ucx
    v = raster_quiv.mesh2d_ucy.values # varn_ucy

    arrow_layer = LayerGroup()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x_start, y_start = x[i, j], y[i, j]
            x_end = x_start + scaling_factor * u[i, j]
            y_end = y_start + scaling_factor * v[i, j]
            arrow = AntPath(
                locations=[(y_start, x_start), (y_end, x_end)],
                dash_array=[10, 20],  # Dashed pattern
                delay=1000,  # Animation speed (lower = faster)
                color="white",
                pulse_color="black"
            )
            arrow_layer.add_layer(arrow)
    for layer in list(current_layer_group.value.layers):
        current_layer_group.value.remove_layer(layer)
    current_layer_group.value.add_layer(scatter_layer) 
    current_layer_group.value.add_layer(arrow_layer)
    plot_curr_map_status.set("Done")
    

In [28]:
@solara.component
def Tab_visu3():
    solara.use_effect(lambda: plot_wl_map_status.set("Idle"), [use_manual_map.value])
    
    with solara.Card("Velocity magnitude [m/s]", style={"width": "100%", "padding": "10px"}):
        
        solara.Switch(label="Manual map file path", value=use_manual_map)
        if use_manual_map.value:
            solara.InputText(label="Path to .nc map file", value=file_nc_map_new, continuous_update=True)
            path_file_nc_map_new = Path(file_nc_map_new.value)
            if file_nc_map_new.value:
                if not file_nc_map_new.value.endswith("_map.nc"):
                    solara.Error(label='Error: File must end with "_map.nc"',text=False,dense=True,outlined=True,icon=False)
                elif not path_file_nc_map_new.exists():
                    solara.Error("File does not exist.")
        
        solara.Button("Create Map",on_click=print_current_map)
        if plot_curr_map_status.value == "Plotting":
            solara.Markdown("Preparing plot... Please wait. This might take a couple of minutes.")
        elif plot_curr_map_status.value == "Done":
            solara.Markdown("**Done!**")


# Tab_visu3()

# FINAL: Putting everything together

In [29]:
model_name = solara.reactive("")
lat_min = solara.reactive(53); lat_max = solara.reactive(54.5); lon_min = solara.reactive(-1.0); lon_max = solara.reactive(1.5) 
date_min = solara.reactive(datetime.date(2013, 12, 1)); date_max = solara.reactive(datetime.date(2013, 12, 2)); ref_date = solara.reactive(datetime.date(2013, 1, 1)) 

def lighten_color(hex_color, amount=0.5):
    hex_color = hex_color.lstrip('#')
    r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    r, g, b = [int(x + (255 - x) * amount) for x in (r, g, b)]
    return f'#{r:02x}{g:02x}{b:02x}'

    
selected_tab = solara.reactive('UserInput') 
current_control = solara.reactive(None)

@solara.component
def SettingsTabs():
    
    solara.use_effect(lambda: selected_tab.set(selected_tab.value), [selected_tab.value])

    def update_map_layers_and_controls():
        for layer in list(m.layers[1:]):
            m.remove_layer(layer)
        current_layer_group.set(tab_layers[selected_tab.value])
        m.add_layer(current_layer_group.value)
        # controls
        if current_control.value is not None and current_control.value in m.controls:
            m.remove_control(current_control.value)
        if selected_tab.value in tab_controls.value:
            control = tab_controls.value[selected_tab.value]
            if control not in m.controls:
                m.add_control(control)
            current_control.set(control)
        else:
            current_control.set(None)
    solara.use_effect(update_map_layers_and_controls, [selected_tab.value, control_update_signal.value]) 
    
    def get_button_style(tab_name, original_color, font_color):
        if selected_tab.value == tab_name:
            return {"background-color": lighten_color(original_color), "color": font_color}
        return {"background-color": original_color, "color": font_color}
        

    with solara.Column(style={"width": "100%", "align-items": "center"}):
        with solara.Sidebar():
            with solara.Column(gap="10px", style={"align-items": "stretch"}):
                solara.Button("1. User Input", on_click=lambda: selected_tab.set('UserInput'), style=get_button_style('UserInput', "#999999","black"))
                solara.Button("2. Grid Generation", on_click=lambda: selected_tab.set('GridGen'), style=get_button_style('GridGen', "#999999","black"), disabled=not userinputdone.value)
                if not gridgendone.value:
                    solara.Markdown("User input needs to be set-up first.")
                solara.Button("3. Boundary Conditions", on_click=lambda: selected_tab.set('Bnd'), style=get_button_style('Bnd', "#999999","black"), disabled=not gridgendone.value)
                if not bnddone.value:
                    solara.Markdown("Grid needs to be generated first.")
                solara.Button("4. Forcings", on_click=lambda: selected_tab.set('Forcing'), style=get_button_style('Forcing', "#999999","black"), disabled=not bnddone.value)
                if not forcingdone.value:
                    solara.Markdown("Boundary conditions need to be set-up first.")
                solara.Button("5. Observations", on_click=lambda: selected_tab.set('Obs'), style=get_button_style('Obs', "#999999","black"), disabled=not forcingdone.value)
                if not obsdone.value:
                    solara.Markdown("Forcing files need to be generated/implemented first.")
                solara.Button("6. Generate Files", on_click=lambda: selected_tab.set('mdu'), style=get_button_style('mdu', "#999999","black"), disabled=not obsdone.value)
                if not mdudone.value:
                    solara.Markdown("Observation points need to be added first.")
                solara.Button("7. RUN MODEL", on_click=lambda: selected_tab.set('run'),style=get_button_style('run', "#FF5733","white"), disabled=not mdudone.value)
                if not rundone.value:
                    solara.Markdown("Running/Input files needs to be generated first.")
                solara.Button("8. Timeseries", on_click=lambda: selected_tab.set('visu1'),style=get_button_style('visu1', "#104e8b","white")) #, style={"background-color": "	#104e8b", "color": "white"})
                solara.Button("9. Water levels", on_click=lambda: selected_tab.set('visu2'),style=get_button_style('visu2', "#104e8b","white")) #, style={"background-color": "	#104e8b", "color": "white"})
                solara.Button("10. Currents", on_click=lambda: selected_tab.set('visu3'),style=get_button_style('visu3', "#104e8b","white")) #, style={"background-color": "	#104e8b", "color": "white"})
    

    if selected_tab.value == 'UserInput':
        Tab_User_Input()
    elif selected_tab.value == 'GridGen':
        Tab_Grid()
    elif selected_tab.value == 'Bnd':
        Tab_Boundary_Cond()
    elif selected_tab.value == 'Forcing':
        Tab_Forcings()
    elif selected_tab.value == 'Obs':
        Tab_Observation()
    elif selected_tab.value == 'mdu':
        Tab_gen_mdu()
    elif selected_tab.value == 'run':
        Tab_run_model()
    elif selected_tab.value == 'visu1':
        Tab_visu1()
    elif selected_tab.value == 'visu2':
        Tab_visu2()
    elif selected_tab.value == 'visu3':
        Tab_visu3()
    elif selected_tab.value == 'xxx':
        Tabxxx()

@solara.component
def Page():
    with solara.Columns():
        with solara.Column(style={"width": "70%", "min-width": "650px"}):
            display(m) #.value) # m.value

        with solara.Column(style={"width": "30%", "min-width": "500px"}):
            SettingsTabs()

Page()


Cannot show ipywidgets in text